In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
from tensorflow_model_optimization.quantization import keras

RecursionError: maximum recursion depth exceeded while calling a Python object

In [5]:
tensorflow_model_optimization(__version__)

NameError: name 'tensorflow_model_optimization' is not defined

In [3]:
import tensorflow as tf
# from tensorflow_model_optimization.python.core.keras.compat import keras
import tensorflow_model_optimization as tfmot

import sys 
sys.path.append("C:/Users/polin/TinyBirdSounds") 

from create_spectrogram_and_time_series_dataset_use import (
    create_spectrograms_from_audio_dataset, 
)
from helper_functions import (
    get_file_size, 
    convert_bytes, 
    convert_prefetchdataset_to_numpy_arrays,
    predict_and_print_full_results,
    evaluate_time_of_prediction
    )

In [6]:
sample_rate = 16000
# length_of_audio = 144000

In [7]:
train_dataset = tf.keras.utils.audio_dataset_from_directory("dataset/training", labels='inferred', sampling_rate=16000, batch_size=32, shuffle=True, seed=42)
test_dataset = tf.keras.utils.audio_dataset_from_directory("dataset/testing", labels='inferred', sampling_rate=16000, batch_size=32, shuffle=True, seed=42)
val_dataset = tf.keras.utils.audio_dataset_from_directory("dataset/validation", labels='inferred', sampling_rate=16000, batch_size=32, shuffle=True, seed=42)


Found 11292 files belonging to 2 classes.
Found 1393 files belonging to 2 classes.
Found 1380 files belonging to 2 classes.


In [8]:
train_spectrogram_ds = create_spectrograms_from_audio_dataset(train_dataset, sample_rate = sample_rate).cache().prefetch(tf.data.AUTOTUNE)
test_spectrogram_ds = create_spectrograms_from_audio_dataset(test_dataset, sample_rate = sample_rate).cache().prefetch(tf.data.AUTOTUNE)
val_spectrogram_ds = create_spectrograms_from_audio_dataset(val_dataset, sample_rate = sample_rate).cache().prefetch(tf.data.AUTOTUNE)

x_train_np, y_train_np = convert_prefetchdataset_to_numpy_arrays(train_spectrogram_ds)
x_val_np, y_val_np = convert_prefetchdataset_to_numpy_arrays(val_spectrogram_ds)
x_test_np, y_test_np = convert_prefetchdataset_to_numpy_arrays(test_spectrogram_ds)

2024-06-18 11:51:15.914539: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 48234496 exceeds 10% of free system memory.
2024-06-18 11:51:16.188886: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 48234496 exceeds 10% of free system memory.
2024-06-18 11:51:16.532472: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 48234496 exceeds 10% of free system memory.
2024-06-18 11:51:17.609459: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 48234496 exceeds 10% of free system memory.
2024-06-18 11:51:17.815471: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 48234496 exceeds 10% of free system memory.


In [10]:
BASE_MODEL_FILE_NAME = "models_from_notebooks/cnn_mel_spec_16kHz_baseline.keras"
base_model = tf.keras.models.load_model(BASE_MODEL_FILE_NAME)
base_model.summary()
convert_bytes(get_file_size(BASE_MODEL_FILE_NAME), "MB")

ValueError: Layer 'conv2d_16' expected 2 variables, but received 0 variables during loading. Expected: ['conv2d_16/kernel:0', 'conv2d_16/bias:0']

In [7]:
quantize_model = tfmot.quantization.keras.quantize_model

# q_aware stands for for quantization aware.
q_aware_model = quantize_model(base_model)

# `quantize_model` requires a recompile.
q_aware_model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

q_aware_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_layer (QuantizeLa  (None, 559, 80, 1)        3         
 yer)                                                            
                                                                 
 quant_conv2d (QuantizeWrap  (None, 557, 78, 3)        39        
 perV2)                                                          
                                                                 
 quant_max_pooling2d (Quant  (None, 139, 19, 3)        1         
 izeWrapperV2)                                                   
                                                                 
 quant_conv2d_1 (QuantizeWr  (None, 137, 17, 8)        243       
 apperV2)                                                        
                                                                 
 quant_max_pooling2d_1 (Qua  (None, 34, 4, 8)          1

In [8]:
q_aware_model.fit(x_train_np[:500], y_train_np[:500],
                  batch_size=32, epochs=1, validation_split=0.1)



15/15 [==============================] - 2s 58ms/step - loss: 0.0209 - accuracy: 0.9867 - val_loss: 6.1587e-04 - val_accuracy: 1.0000


In [9]:
_, baseline_model_accuracy = base_model.evaluate(
    x_test_np, y_test_np, verbose=0)

_, q_aware_model_accuracy = q_aware_model.evaluate(
   x_test_np, y_test_np, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)
print('Quant test accuracy:', q_aware_model_accuracy)

Baseline test accuracy: 0.9906676411628723
Quant test accuracy: 0.9885140061378479


In [10]:
converter = tf.lite.TFLiteConverter.from_keras_model(q_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

quantized_tflite_model = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\polin\AppData\Local\Temp\tmp4k4q4v6_\assets


INFO:tensorflow:Assets written to: C:\Users\polin\AppData\Local\Temp\tmp4k4q4v6_\assets
C:\Users\polin\AppData\Roaming\Python\Python39\site-packages\tensorflow\lite\python\convert.py:983: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


In [11]:
print("Validation dataset:")
(
    y_pred_val, 
    non_overlap_patritions_f1_scores_val, 
    bootstrap_patritions_f1_scores_val,
    ) = predict_and_print_full_results(quantized_tflite_model, x_val_np, y_val_np, model_format="tf_lite")

print("\nTest dataset:")
(
    y_pred_test, 
    non_overlap_patritions_f1_scores_test, 
    bootstrap_patritions_f1_scores_test,
    ) = predict_and_print_full_results(quantized_tflite_model, x_test_np, y_test_np, model_format="tf_lite")

Validation dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 99.20%
Recall: 99.04%
Precision: 99.15%
F1-score: 98.83%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.987996971158965

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9893040888731458

Test dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 98.92%
Recall: 98.81%
Precision: 98.26%
F1-score: 98.36%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9832425334424808

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9842150230161378


In [12]:
print("For Test dataset:")
evaluate_time_of_prediction(quantized_tflite_model, x_test_np, y_test_np, model_format="tf_lite")

For Test dataset:
Accuracy: 98.92%
Recall: 98.81%
Precision: 98.26%
F1-score: 98.36%

Time to make a prediction for a single data point
Mean: 0.004 seconds
Max: 0.009 seconds
Min: 0.003 seconds


In [13]:
# Save model and print its size
QAT_MODEL_FILE_NAME = "models_from_notebooks/cnn_spectrogram_qat.tflite"
open(QAT_MODEL_FILE_NAME, "wb").write(quantized_tflite_model)
print("Model file name: ", QAT_MODEL_FILE_NAME)
convert_bytes(get_file_size(QAT_MODEL_FILE_NAME), "KB")

Model file name:  models_from_notebooks/cnn_spectrogram_qat.tflite
File size: 39.711 Kilobytes
